# Training

This notebook consists on some code for training a neural network on the ECG dataset, to the aggregated superclass level ("NORM", "MI", "STTC", "CD", "HYP").

This notebook uses the scripts 
- *train.py* to define the training function (quite simple; could be just defined in this notebook)
- *trainutils.py* defining some utils related to the training 
- *models.py* defining the 1d-resnet model used

## Mount repository
Doing this to make sure we have all the files of the repository.
Needed to import the *train.py*, *trainutils.py* and *models.py* files.

In [ ]:
!git clone https://[PERSONAL ACCESS TOKEN REMOVED HERE!!!]@github.com/sergi-andreu/Idoven-challenge.git idoven

fatal: destination path 'idoven' already exists and is not an empty directory.


## Mount drive and import some required packages

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
from scipy.fft import rfft, rfftfreq
import scipy.signal as sp

import seaborn as sb
import pandas as pd

import torch
from torch.utils.data import TensorDataset

import os

from sklearn.preprocessing import StandardScaler

### Set the seed for reproducibility

In [ ]:
def set_all_seeds(seed):
  np.random.seed(seed)
  torch.manual_seed(seed)
  torch.cuda.manual_seed(seed)
  torch.backends.cudnn.deterministic = True

set_all_seeds(0)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Import the training and test data

Using folds 1-8 for training, 9 for testing. Fold 10 is left for evaluation

In [ ]:
drivepath = "drive/MyDrive/idoven"
fold_idx = 1
X = np.load(f"{drivepath}/nparrays/{str(fold_idx).zfill(2)}.npy")
Y = np.load(f"{drivepath}/nparrays/labels/{str(fold_idx).zfill(2)}.npy")

for fold_idx in range(2, 9):
  X = np.append(X, np.load(f"{drivepath}/nparrays/{str(fold_idx).zfill(2)}.npy"), axis=0)
  Y = np.append(Y, np.load(f"{drivepath}/nparrays/labels/{str(fold_idx).zfill(2)}.npy"), axis=0)


def apply_scaler(X, scaler):
    X_tmp = []
    for x in X:
        x_shape = x.shape
        X_tmp.append(scaler.transform(x.flatten()[:,np.newaxis]).reshape(x_shape))
    X_tmp = np.array(X_tmp)
    return X_tmp

# Scale the data using a standard scaler
scaler = StandardScaler()
scaler.fit(np.vstack(X).flatten()[:,np.newaxis].astype(float))
X = apply_scaler(X, scaler)

X, Y = torch.from_numpy(X).float(), torch.from_numpy(Y).float()
train_dataset = TensorDataset(X,Y)
del X, Y

X_test = np.load(f"{drivepath}/nparrays/09.npy")
Y_test = np.load(f"{drivepath}/nparrays/labels/09.npy")

X_test = apply_scaler(X_test, scaler)

X_test, Y_test = torch.from_numpy(X_test).float(), torch.from_numpy(Y_test).float()
test_dataset = TensorDataset(X_test, Y_test)
del X_test, Y_test

Save the scaler (for later evaluation)

In [ ]:
import pickle as pck
dict_scaler = {"scale_" : scaler.scale_, "mean_" : scaler.mean_, "var_" : scaler.var_}

with open("drive/MyDrive/idoven/scalers/standardscaler.txt","wb") as filehandler:
  pck.dump(dict_scaler,filehandler)

### Define the dataloaders

In [ ]:
from torch.utils.data import DataLoader, BatchSampler

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

### Import utils

In [ ]:
from idoven.models import *
from idoven.trainutils import *
from idoven.train import *

In [ ]:
from sklearn.metrics import roc_auc_score, average_precision_score

loss_meter = AverageMeter()
CEloss = torch.nn.CrossEntropyLoss()

### Create model

In [ ]:
num_outputs = 5
model = resnet18(num_outputs = num_outputs).to(device)

# Import Weights&Biases
This is used for experiment tracking. Here, Weights&Biases is specially useful since the colab notebook sometimes break, and it is a way to log the data on the cloud, and access is later.

In [ ]:
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.9 MB 13.6 MB/s 
     |████████████████████████████████| 166 kB 59.4 MB/s 
     |████████████████████████████████| 182 kB 71.1 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
     |████████████████████████████████| 166 kB 68.3 MB/s 
     |████████████████████████████████| 162 kB 76.2 MB/s 
     |████████████████████████████████| 162 kB 72.0 MB/s 
     |████████████████████████████████| 158 kB 76.1 MB/s 
     |████████████████████████████████| 157 kB 75.2 MB/s 
     |████████████████████████████████| 157 kB 74.5 MB/s 
     |████████████████████████████████| 157 kB 74.2 MB/s 
     |████████████████████████████████| 157 kB 66.2 MB/s 
     |████████████████████████████████| 157 kB 78.5 MB/s 
     |████████████████████████████████| 157 kB 62.6 MB/s 
     |████████████████████████████████| 157 kB 67.4 MB/s 
     |███████████████████████████

### Create a wandb sweep for hyperparameter tuning
I have tried also other parameters (not doing an exhaustive grid search). Only doing a grid search here on the learning rate values.

In [ ]:
import wandb

sweep_config = {
  "name": "Learning rate sweep 2",
  "method": "grid",
  "parameters": {
        "lr": {
            "values": [1e-5, 5e-5, 1e-4, 5e-4, 1e-3]
        },
        "epochs" : {
            "values" : [5]
        },
        "log_every" : {
            "values" : [10]
        },
        "model_used" : {
            "values" : ["ResNet18"]
        }
    }
}

# Define the sweep ID
sweep_id = wandb.sweep(sweep_config, entity="sergi-andreu", project="idoven")

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Create sweep with ID: g0nr5xgv
Sweep URL: https://wandb.ai/sergi-andreu/idoven/sweeps/g0nr5xgv


### Train the model

The model is trained by the sweep (different trains, with the different parameters of the sweep).

For that matter, we define a TRAIN() function on the train() function, that mounts the model (the model is not defined on the train() function.

In [ ]:
save_loc = "drive/MyDrive/idoven/models/"

def TRAIN():
  model = resnet18(num_outputs = num_outputs).to(device)
  #model = resnet34(num_outputs = num_outputs).to(device)
  train(model, train_dataloader, test_dataloader, verbose=False, 
        save_model=True, save_loc=save_loc)



Run the sweep

In [ ]:
wandb.agent(sweep_id, function=TRAIN)

wandb: Agent Starting Run: hzcofnd0 with config:
wandb: 	epochs: 5
wandb: 	log_every: 10
wandb: 	lr: 1e-05
wandb: 	model_used: ResNet18
ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: sergi-andreu. Use `wandb login --relogin` to force relogin


CD auc,▂▂▁▃▄▅▆▆▇▆▇▇▇▇▇▇▇▇█▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇██▇
CD auprc,▂▂▁▂▃▄▅▆▇▆▇▇▇▇███▇█▇▇▇█▇▇████▇▇████▇▇███
HYP auc,▁▁▃▄▅▅▅▅▅▆▆▆▆▆▇▇▆▇▇▇▇▇▇▇▇▇▇██▇▇▇█████▇██
HYP auprc,▁▁▂▃▄▃▄▅▅▅▆▇▇▇▇▇▆▇▇▇▇▇▇▇▇▇███▆▇▇█████▆██
MI auc,▁▁▁▃▄▅▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇█▇▇██▇██▇
MI auprc,▁▁▁▂▃▄▅▆▆▆▆▆▆▇▆▇▇▇▇▇▇▇▇▇▇█▇▇█▇▇████████▇
NORM auc,▁▁▃▅▇▇▇▇▇███████████████████████████████
NORM auprc,▁▁▃▅▇▇▇▇████████████████████████████████
STTC auc,▁▂▃▂▅▅▆▆▆▇▇▇▇█▇▇██▇▇████████████████████
STTC auprc,▁▁▂▂▄▄▄▅▅▅▆▆▇▇▇▇▇▇▇▇▇▇▇▇█████▇██████████
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████


wandb: Agent Starting Run: xhnl3t0t with config:
wandb: 	epochs: 5
wandb: 	log_every: 10
wandb: 	lr: 5e-05
wandb: 	model_used: ResNet18
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


CD auc,▂▄▄▅▄▃▅▆▅▆▆▇▅▃▆▆▇▇▅▄▅▆▄▇█▇▇▆▇▅▄▄▁▇▅▅█▇▆▆
CD auprc,▂▃▄▅▄▂▅▆▅▆▇▇▅▃▆▆▇▇▆▄▅▇▄▇█▇▇▆▇▄▄▂▁▆▄▅█▆▅▅
HYP auc,▁▃▅▅▆▆▇▆▆▇▇▆▆▇▇█▇▆▇▅▆█▇▇▇▇▆▇▇▇▆▇▅▆▆▆▇▇▆▇
HYP auprc,▁▃▃▄▄▅▅▆▅▆▆▆▄▇▇▇▇▅▇▄▆█▆▆▇▇▆▆▇▇▆▆▅▇▇▇▇▇▆▇
MI auc,▁▅▅▇▆█▇▇▆▇▆▆▃▇▇▆████▄▆▆▇▇█▇▇▆▆▇█▇▇▆▇▆▇█▇
MI auprc,▁▄▄▆▇▇▆▆▆▆▅▆▂▆▆▅█▇▇█▃▅▅▆▇▇▇▇▅▅▇▇▇▆▆▆▅▇█▇
NORM auc,▁▆▇▇▇█▇█████████████████████████████████
NORM auprc,▁▆▇█▇███████████████▇███████▇██████▇████
STTC auc,▁▄▆▆██▇████████▇▇███████▇███████▇███▇▇▇█
STTC auprc,▁▃▅▅▇▆▇█▇▇▇▇▇█▇▇▇█▇█▇███▆▇█▇▇█▇▇▇▇▇▇▇▆▇▇
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████


wandb: Agent Starting Run: aj6halk1 with config:
wandb: 	epochs: 5
wandb: 	log_every: 10
wandb: 	lr: 0.0001
wandb: 	model_used: ResNet18
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


...

wandb: Agent Starting Run: e2uu0vhg with config:
wandb: 	epochs: 5
wandb: 	log_every: 10
wandb: 	lr: 0.0005
wandb: 	model_used: ResNet18
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


CD auc,▁▄▅▅▅▅▆▄▆▆▅▇▆▇▆▇▄▇▃▆▇▆▅█▇███▇█▇█▇███▇▇█▇
CD auprc,▁▄▄▅▆▆▅▅▆▆▆▆▆█▆▇▆▇▃▆█▅▅█▇▇▇█▇█▇█▆▇█▇▇▇▇▇
HYP auc,▁▁▃▄▅▆▃▅▅▅▅▆▅▅▅▄▆▆▇▆▆▆▅▆▇▇▇▇▅▄▆▇▇▆▆█▇▇▇▇
HYP auprc,▁▁▂▄▅▄▂▃▅▆▄▄▃▃▄▂▅▅▇▅▆▄▄▅▇▆▆▅▂▃▅█▅▃▃▇▆▄▄▅
MI auc,▁▃▃▃▅▄▅▅▄▄▇▆▃▅▄▆▆▆▇▆▅▇▇▅▆▇▆▇█▇▆▇▆▆▇▅██▇▅
MI auprc,▁▂▃▃▄▄▃▃▃▅▇▄▂▅▄▄▆▅▅▅▅▇▅▅▅▆▆██▅▅▇▄▅▆▄█▇▆▄
NORM auc,▁▅▆▆▇▆▇▇▆█▇▇▇██▇███▇██▇▇████▇██▇████████
NORM auprc,▁▄▆▆▇▆▇▇▆█▇▇▇▇█▇▇▇▇▇▇█▇▇██▇▇▇▇█▇▇███▇▇██
STTC auc,▁▄▅▅▇▇▇█▆█▇▇██▇▆▇▇▇▇▇█▇▇▇▇█▆▇██▆▇▆█▇▇██▇
STTC auprc,▁▃▄▅▆▇▇▇▅█▇▅█▇▇▅▇▆▇█▇█▇▇▇▆▇▆▆▇▇▅▇▅█▇▇█▇▆
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████


wandb: Agent Starting Run: sf6b0136 with config:
wandb: 	epochs: 5
wandb: 	log_every: 10
wandb: 	lr: 0.001
wandb: 	model_used: ResNet18
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


CD auc,▄▂▃▅▅▅▄▁▆▅▅▆▄▇▅▇▇▅▆▅▇▇▅▅▄▇▇▆▇▅▇▅▅█▆▅▇▇▆▆
CD auprc,▃▂▃▅▆▅▅▁▆▅▆▆▄▇▅▇▇▅▅▄▆▇▆▃▄▆▇▆▆▅▆▄▄█▆▄▇▆▅▅
HYP auc,▁▅▅▆▆▇▆▆▇▅▇▇▇▆▇▆▇▆▇▇▇▇▇▇▇▇▇▇▇█▇█▇▇██▇▇▇█
HYP auprc,▁▃▄▅▄▇▄▅▄▃▆▄▄▄▅▄▅▄▆▄▅▅▅▆▅▄▄▅▄█▅▅▅▆▆▅▇▄▄▆
MI auc,▃▁▆▆▄▄▇▇▂▇▆▇▆▆▇▄▅▇▆▇▇▇▆▇█▆▆▇▅▇▇██▇▇▇█▆█▇
MI auprc,▁▁▅▄▃▃▅▅▁▄▄▄▄▄▄▃▄▇▄▅▅▆▄▅▆▅▄▅▃▆▅█▇▅▄▅█▄▆▅
NORM auc,▁▅▇▇▇▇▇▇▇▇████▇▇████████████████████████
NORM auprc,▁▄▇▆▇▇▇▆▇▆▇▇██▇▇███████████▇██▇████████▇
STTC auc,▁▆▇█▇█▇▇█▇▇██▇█▇█▇▇██▇▇█████▇████▇▇█████
STTC auprc,▁▄▆▇▇█▆▇█▆▆█▇▆█▆▇▆▅▇▇▅▆▇▇▇▇█▅▇▇▇▇▆▆█▇█▇▇
epoch,▁▁▁▁▁▁▁▁▃▃▃▃▃▃▃▃▅▅▅▅▅▅▅▅▆▆▆▆▆▆▆▆████████


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


The results are now stored in my Wandb project. Some plots may be added in the readme file of the repository.